In [ ]:
import sys
!{sys.executable} -m pip install --upgrade pip
!{sys.executable} -m pip install pandas pycountry matplotlib statsmodels
;

  Obtaining dependency information for pip from https://files.pythonhosted.org/packages/50/c2/e06851e8cc28dcad7c155f4753da8833ac06a5c704c109313b8d5a62968a/pip-23.2.1-py3-none-any.whl.metadata
Using cached pip-23.2.1-py3-none-any.whl (2.1 MB)
  Attempting uninstall: pip
    Found existing installation: pip 22.0.4
    Uninstalling pip-22.0.4:
      Successfully uninstalled pip-22.0.4


''

# Import and clean KIO data

## Import KIO csv files

Import the raw KIO datasets and do simple preparsing (e.g., remove spaces from column names, strip whitespace from fields, convert some fields to lowercase to make enumerated values consistent).

## Standardize columns

- Unify slightly different naming conventions ('sub-region' vs 'sub_region')
- Remove columns that are redundant (remove 'year' because it is reduntant with 'start_time'/'end_time'), are indices w/ collisions across years ('id'), or are not particularly informative/useful ('latitude', 'longitude')
- Rename some columns for clarity ("area_name" -> "areas_affected")

In [ ]:
import pandas
pandas.set_option('display.max_colwidth', None)
pandas.set_option('display.max_columns', None)  

kio_filenames = { "2018" : "../data/kio/kio_2016-2018.csv",
                  "2019" : "../data/kio/kio_2019.csv",
                  "2020" : "../data/kio/kio_2020.csv",
                  "2021" : "../data/kio/kio_2021.csv"
                }


###################################################################################################

def kio_preprocessing(df):
    """"
    This function applies the first pass of simple transformations to the 
    dataset. 
    Current steps include:
        - Remove leading/trailing whitespace from column names
        - Replace spaces in column names
        - Strip leading/trailing whitespace from entries where applicable
        - Convert enumerated fields to lowercase
        - Replace characters like '&' with ' ' 
    See code for specifics on which steps are performed on which fields
    """
    df.columns = df.columns.str.lower()
    df.columns = df.columns.str.replace(' ','')
    
    fields_to_strip = {
            'info_source', 'news_link', 'continent', 'fullorservice-based', 
            'shutdown_type_new', 'decision_maker', 
            'affected_network', 'sub_region', 'country',
            'fullorservice-based',
            'sub_region', 'country', 'geo_scope', 'area_name', 'ordered_by',
            'shutdown_type_group', 'shutdown_type', 'official_just', 
            'actual_cause', 'services_affected', 'service_details',
            'other_service_details(specify)', 'sms_and_phone_call_affected',
            'telcos_involved', 'gov_ack', 'other_just_details', 
            'off_statement', 'other_cause_details', 'election',
            'violence', 'hr_abuse_reported', 'users_notified',
            'users_affected/targetted', 'legal_justif', 'legal_method',
            'telco_resp', 'telco_ack', 'state/india',
            'sms_affected', 'phone_call_affected',
        }
    
    fields_to_lower = {
            'info_source', 'fullorservice-based', 'decision_maker', 
            'shutdown_type_new', 'fullorservice-based',
            'affected_network', 'geo_scope', 'ordered_by',
            'shutdown_type_group',
            'shutdown_type', 'official_just', 'actual_cause',
            'services_affected', 'service_details',
            'sms_and_phone_call_affected', 'gov_ack',
            'other_cause_details', 'election',
            'violence', 'hr_abuse_reported', 'users_notified',
            'users_affected/targetted', 'legal_method', 'telco_resp',
            'shutdown_extent', 'start_date_type', 'shutdown_status'
        }
    
    field_names_replace = {
            "services_affected" : [('&', ' ')],
            'telcos_involved' : [('\n', ' ')],
            'geo_scope' : [('  ', ' ')]
        }
    
    for field in df.columns:
        if field in fields_to_strip:
            df[field] = df[field].str.strip()
        if field in fields_to_lower:
            df[field] = df[field].str.lower()
        if field in field_names_replace:
            for find, replace in field_names_replace[field]:
                df[field] = df[field].str.replace(find, replace)                  
    return df

kio_df_split = {}
for year in kio_filenames:
    kio_df_split[year] = kio_preprocessing(pandas.read_csv(kio_filenames[year], skipinitialspace = True))
    kio_df_split[year] = kio_df_split[year].assign(kio_source_year=year)
    print("KIO Year: {} | # of rows: {}".format(year, len(kio_df_split[year])))

del_cols = ['year', 'latitude', 'longitude', 'id']
rename_cols = [("area_name", "areas_affected"), ('sub-region', 'sub_region')]

for year in kio_df_split:
    for find, replace in rename_cols:
        if find in kio_df_split[year]:
            kio_df_split[year][replace] = kio_df_split[year][find]
            kio_df_split[year] = kio_df_split[year].drop(columns=find)
    for col in del_cols:
        if col in kio_df_split[year]:
            kio_df_split[year] = kio_df_split[year].drop(columns=col)

KIO Year: 2018 | # of rows: 373
KIO Year: 2019 | # of rows: 213
KIO Year: 2020 | # of rows: 155
KIO Year: 2021 | # of rows: 182


## KIO data field change summary

Print information about the year-to-year changes in the fields available in the KIO datasets

In [ ]:
print("Columns unique to 2016-2018 [x ∈ (2018 - 2019 - 2020)]:")
for field in sorted(list(  set(kio_df_split['2018'].columns) 
                         - set(kio_df_split['2019'].columns)
                         - set(kio_df_split['2020'].columns))):
    print("\t{}".format(field))

years = sorted(kio_df_split)
for year1, year2 in zip(years[:-1], years[1:]):
    print("\nColumns added in {} [x ∈ ({} - {})]:".format(year2, year2, year1))
    for field in sorted(list(  set(kio_df_split[year2].columns)
                             - set(kio_df_split[year1].columns))):
        print("\t{}".format(field))

    print("\nColumns removed in {} [x ∈ ({} - {})]:".format(year2, year1, year2))
    for field in sorted(list(  set(kio_df_split[year1].columns)
                             - set(kio_df_split[year2].columns))):
        print("\t{}".format(field))

Columns unique to 2016-2018 [x ∈ (2018 - 2019 - 2020)]:
	duration_days
	duration_hours
	service_details
	services_affected
	shutdown_type
	shutdown_type_group

Columns added in 2019 [x ∈ (2019 - 2018)]:
	affected_network
	decision_maker
	duration
	election
	facebook_affected
	fullorservice-based
	gov_ack
	hr_abuse_reported
	instagram_affected
	legal_justif
	legal_method
	off_statement
	other_cause_details
	other_just_details
	other_service_details(specify)
	shutdown_type_new
	sms_and_phone_call_affected
	telco_ack
	telco_resp
	telcos_involved
	telegram_affected
	twitter_affected
	users_affected/targetted
	users_notified
	violence
	whatsapp_affected

Columns removed in 2019 [x ∈ (2018 - 2019)]:
	duration_days
	duration_hours
	service_details
	services_affected
	shutdown_type
	shutdown_type_group

Columns added in 2020 [x ∈ (2020 - 2019)]:
	phone_call_affected
	sms_affected
	state/india

Columns removed in 2020 [x ∈ (2019 - 2020)]:
	sms_and_phone_call_affected
	sub_region

Columns added 

## Process & clean the KIO data

- Parse event start/end date strings, convert to datetime objects
- Unify "geo_scope" convention ("local", "regional", "national")
- Create boolean columns for event categories (shutdown, throttle, service-based)
- Create boolean columns for types of networks affected (broadband, mobile)


In [ ]:
###################################################################################################
# Parse timestamps using dateutil.parser(date_str, fuzzy=True)

import datetime
import dateutil.parser as dparser
import numpy

def parse_date_kio(ts):
    if (type(ts) == datetime.datetime or 
            type(ts) == type(None) or 
            pandas.core.dtypes.common.is_datetime_or_timedelta_dtype(ts)):
        return ts
    elif type(ts) == float:
        return None
    ts = ts.strip().lower()
    if (ts == "ongoing" or 
            ts == "unknown" or ts == "unkown"):
        return None
    try:
        return dparser.parse(ts, fuzzy=True)
    except ValueError:
        print("Unable to parse a row, skipping row with bad date ('{}')".format(ts.replace("\n", " ")))
        return None
    
def parse_dates_kio(df):
    df['start_date'] = df['start_date'].transform(parse_date_kio)
    df['end_date'] = df['end_date'].transform(parse_date_kio) 
    return df

for year in kio_df_split:
    kio_df_split[year] = parse_dates_kio(kio_df_split[year])

###################################################################################################
# Unify geo_scope convention across datasheets
#
# Output enumerated values: {"local", "regional", "national"}
# 
# For 2019, 2020 map:
#     "level 1" -> "local"
#     "level 2" -> "regional"
#     "level 3" -> "national"
#
# For 2021 map:
#     "it only affected one city, county, or village" -> "local"
#     "it affected more than one city in the same state, province, or region" -> "region"
#     "it affected locations in more than one state, province, or region" -> "national"
#     "not sure" -> None

def unify_geo_scope(geo_scope):
    if type(geo_scope) == type(None):
        return geo_scope
    if type(geo_scope) != str and numpy.isnan(geo_scope):
        return None
    if geo_scope == 'multi-regional':
        return "national"
    elif geo_scope == 'level 1':
        return "local"
    elif geo_scope == "level 2":
        return "regional"
    elif geo_scope == "level 3":
        return "national"
    elif geo_scope == "it only affected one city, county, or village":
        return "local"
    elif geo_scope == "it affected more than one city in the same state, province, or region":
        return "regional"
    elif geo_scope == "it affected locations in more than one state, province, or region":
        return "national"
    elif geo_scope == "not sure":
        return None
    return geo_scope

for year in kio_df_split:
    kio_df_split[year]['geo_scope'] = kio_df_split[year]['geo_scope'].transform(unify_geo_scope)

###################################################################################################
# Label event type (true/false) each in three categories: 
#   - shutdown
#   - throttling
#   - service-based

def label_network_shutdown(row):
    if 'shutdown_type_new' in row:
        # In 2019/2020 format
        if ('shutdown' in row['shutdown_type_new'] 
                and 'full' in row['fullorservice-based']):
            return True
        else:
            return False
    elif 'shutdown_type_group' in row:
        # In pre-2019 format
        if (row['shutdown_type_group'] == 'service-based' or 
                'service-based' in row['shutdown_type'] or
                (type(row['services_affected']) == str and
                 'service-based' in row['services_affected'])):
            return False
        elif row["shutdown_type_group"] == "throttling":
            return False
        else:
            return True
    else:
        # In 2020/2021 format
        if (('shutdown' in row['shutdown_type']) and 
                ('full network' in row['shutdown_extent'])):
            return True
        else:
            return False


def label_throttling(row):
    if 'shutdown_type_new' in row:
        # In 2019/2020 format
        if "throttle" in row["shutdown_type_new"]:
            return True
        else:
            return False
    elif 'shutdown_type_group' in row:
        # In pre-2019 format
        if row["shutdown_type_group"] == "throttling":
            return True
        else:
            return False
    elif 'shutdown_extent' in row:
        # In 2020/2021 format
        if 'throttle' in row['shutdown_type']:
            return True
        else:
            return False
        


def label_service_based(row):
    if 'fullorservice-based' in row:
        # In 2019/2020 format
        if 'service-based' in row['fullorservice-based']:
            return True
        else:
            return False         
    elif 'shutdown_type_group' in row:
        # In pre-2019 format
        if (row['shutdown_type_group'] == 'service-based' or 
                'service-based' in row['shutdown_type'] or
                (type(row['services_affected']) == str and
                    'service-based' in row['services_affected'])):
            return True
        else:
            return False
    elif 'shutdown_extent' in row:
        # 2020/2021
        if 'service-based' in row['shutdown_extent']:
            return True
        else:
            return False


for year in kio_df_split:
    kio_df_split[year]['full_network_shutdown'] = kio_df_split[year].apply(
        lambda row: label_network_shutdown(row), axis=1)
    kio_df_split[year]['throttling'] = kio_df_split[year].apply(
        lambda row: label_throttling(row), axis=1)
    kio_df_split[year]['service_based'] = kio_df_split[year].apply(
        lambda row: label_service_based(row), axis=1)


################################################################################
# Label networks affected (true/false) in each of these categories: 
#   - broadband
#   - mobile

def label_mobile(row):
    if 'affected_network' not in row:
        # pre-2019
        if ('mobile' in row['shutdown_type_group'] or
                'mobile' in row['shutdown_type_group'] or 
                (type(row['services_affected']) == str and
                      'mobile' in row['services_affected'])):
            return True
        else:
            return False
    else:
        # 2019/2020 or 2020/2021
        if 'mobile' in row['affected_network']:
            return True
        else:
            return False  

def label_broadband(row):
    if 'affected_network' not in row:
        # pre-2019
        if ('broadband internet' in row['shutdown_type_group'] or
                'broadband internet' in row['shutdown_type_group'] or 
                (type(row['services_affected']) == str and
                      'broadband internet' in row['services_affected'])):
            return True
        else:
            return False
    else:
        # 2019/2020 or 2020/2021
        if 'broadband' in row['affected_network']:
            return True
        else:
            return False

for year in kio_df_split:
    kio_df_split[year]['mobile_affected'] = kio_df_split[year].apply(
        lambda row: label_mobile(row), axis=1)
    kio_df_split[year]['broadband_affected'] = kio_df_split[year].apply(
        lambda row: label_broadband(row), axis=1)

Unable to parse a row, skipping row with bad date ('44166')


## Merge KIO datasets, apply fixes, remove duplicates

- _Merge:_ Datasets were split by source file since different time periods use different conventions. Now that the data has been fully processed, cleaned, and standarized, we're ready to merge the data.

- _Fixes:_ There are some mistakes in the original KIO datasets. If we're using a KIO file where the fixes have not yet been applied, apply those fixes here (these fixes were verified with AccessNow).

- _Remove duplicates:_ Some outages that span multiple years are duplicated across datasets. Detect these and remove. Some of this was done by manually modifying the CSV files used as input.


In [ ]:
kio_df = pandas.concat([kio_df_split[year] for year in kio_df_split], axis=0, ignore_index=True).sort_values(by='start_date')

updates = [{"selection" : (  (kio_df['country'] == 'Syria') 
                           & (kio_df['start_date'] == datetime.datetime(2018, 5, 27))
                           & (kio_df['end_date'] == datetime.datetime(2018,5,28))),
            "update_col" : "end_date",
            "update_val" : datetime.datetime(2018,6,12)},
           {"selection" : (  (kio_df['country'] == 'Syria') 
                           & (kio_df['start_date'] == datetime.datetime(2019, 6, 13))
                           & (kio_df['end_date'] == datetime.datetime(2019,6,30))),
            "update_col" : "start_date",
            "update_val" : datetime.datetime(2019,6,9)}
          ]

for update in updates:
    if update['selection'].any():
        kio_df.loc[update['selection'], update['update_col']] = update['update_val']

# IODA data

## Load and process/clean IODA data

- Simplify column names (convert to lowercase, use underscores instead of spaces, make column names, etc.)
- Replace problematic characters/substrings in country names (breaks standardization)
- Add boolean columns for data sources (BGP, AP, and DN)
- Parse timestamp strings to datetime objects (converted to pandas timestamps)

In [ ]:
ioda_filename = "../data/ioda/ioda_investigated_outages_cleaned_phase1+2.csv"

def ioda_preprocessing(df):
    """"
    This function applies the first pass of simple transformations to the 
    dataset. 
    Current steps include:
        - Remove leading/trailing whitespace from column names
        - Replace spaces in column names
        - Strip leading/trailing whitespace from entries where applicable
        - Convert enumerated fields to lowercase
        - Replace characters like '&' with ' ' 
    See code for specifics on which steps are performed on which fields
    """
    cols_to_rename = {"Begin time" : "start_time",
                      "Scope (based on granularity at which the outage appears in IODA)" : "scope",
                      "Detected by IODA?" : "detected_by_ioda",
                      "IODA BGP visible by human" : "BGP",
                      "IODA AP visible by human " : "AP",
                      "IODA IBR visible by human" : "IBR",
                      "AS" : "asn"
                     }
    df = df.rename(columns=cols_to_rename)
    df.columns = df.columns.str.lower()
    df.columns = df.columns.str.replace(' ','_')
    
    fields_to_strip = {}
    fields_to_lower = {}
    fields_to_upper = {"bgp", "ap", "ibr"}
    fields_with_replace = {"country" : [("&", "and")]}
    
    for field in df.columns:
        if field in fields_to_strip:
            df[field] = df[field].str.strip()
        if field in fields_to_lower:
            df[field] = df[field].str.lower()
        if field in fields_to_upper:
            df[field] = df[field].str.upper()
        if field in fields_with_replace:
            for find, replace in fields_with_replace[field]:
                df[field] = df[field].str.replace(find, replace)
        if 'unnamed' in field:
            df = df.drop(columns=field)
    return df

ioda_df = ioda_preprocessing(pandas.read_csv(ioda_filename, skipinitialspace=True))
ioda_df["bgp_alert"] = ioda_df['bgp'].str.contains("TRUE")
ioda_df["ap_alert"] = ioda_df['ap'].str.contains("TRUE")
ioda_df["dn_alert"] = ioda_df['ibr'].str.contains("TRUE")

import dateutil 

def parse_timestamp_ioda(ts):
    if (type(ts) == datetime.datetime or 
            type(ts) == type(None) or 
            type(ts) == float or
            pandas.core.dtypes.common.is_datetime_or_timedelta_dtype(ts)):
        return ts    
    if ": " in ts:
        ts = ts.replace(": ", ":00")
    ts = ts.strip()
    try:
        return dparser.parse(ts, fuzzy=True)
    except (ValueError, pandas.errors.ParserError, dateutil.parser.ParserError):
        print("Unable to parse a row, skipping ('{}')".format(ts.replace("\n", " ")))
        return None 

def parse_dates_ioda(df):
    df['start_time'] = df['start_time'].transform(parse_timestamp_ioda)
    df['end_time'] = df['end_time'].transform(parse_timestamp_ioda) 
    return df

def extract_region(row):
    if 'Region' in row['scope']:
        return row['scope'][row['scope'].find('(')+1:row['scope'].find(')')]
    else:
        return None

def extract_as(row):
    if 'AS' in row['scope']:
        return row['scope'][row['scope'].find('(')+1:row['scope'].find(')')]
    else:
        return None

def update_scope(row):
    if 'Region' in row['scope']:
        return 'Region'
    if 'AS' in row['scope']:
        return 'AS'
    else:
        return row['scope']


ioda_df = parse_dates_ioda(ioda_df)
ioda_df = ioda_df.dropna(subset=["start_time"]).sort_values(by='start_time').reset_index(drop=True)

ioda_df['region'] = ioda_df.apply(lambda row: extract_region(row), axis=1)
ioda_df['asn'] = ioda_df.apply(lambda row: extract_as(row), axis=1)
ioda_df['scope'] = ioda_df.apply(lambda row: update_scope(row), axis=1)

cols_to_drop = ["bgp", "ap", "ibr", "notes", "manually_verified"]

ioda_df = ioda_df.drop(cols_to_drop, axis=1)

# Merge IODA and KIO

- Standardize country names in KIO & IODA datasets
- Match events between KIO and IODA
- Save merged dataset

In [ ]:
import pycountry

tricky_country_name_fixes = {
                         "kyrgyz republic (kyrgyzstan)" : "kyrgyzstan",
                         "united states of america (usa)" : "united states of america",
                         "swaziland" : "Eswatini",
                         "israel and the occupied territories" : "israel",
                         "ivory coast (cote d'ivoire)" : "cote d'ivoire",
                         "myanmar/burma" : "myanmar",
                         "cape verde" : "Republic of Cabo Verde",
                         "timor leste" : "Timor-Leste",
                         "palestinian territories" : "palestine",
                         "laos" : "Lao People's Democratic Republic",
                         "democratic republic of (drc)" : "Congo, The Democratic Republic of the",
                         "curacao" : "Curaçao",
                         "iran" : "Iran, Islamic Republic of",
                         "democratic republic of the congo" : "Congo, The Democratic Republic of the",
                         "srilanka" : "Sri Lanka",
                         "iran (islamic republic of)" : "Iran, Islamic Republic of",
                         "venezuela (bolivarian republic of)" : "Venezuela",
                         "democraticrepubliccongo" : "Congo, The Democratic Republic of the",
                         "reunion" : "Réunion",
                         "bonaire/sint eustatius/saba" : "Bonaire, Sint Eustatius and Saba"
                        }

def get_country_object(country):
    if "NANOG" in country or "puck.nether" in country:
        print(country)
        return None
    if country.lower() in tricky_country_name_fixes:
        country = tricky_country_name_fixes[country.lower()]
    try:
        return pycountry.countries.lookup(country)
    except LookupError:
        try:
            found = pycountry.countries.search_fuzzy(country)
            if len(found) > 1:
                print("Using first country in fuzzy search results for:", country)
                for item in found:
                    print("\t",item)
            return found[0]
        except LookupError:
            print("Skipping row for which a country could not be identified:", country)
            return None

def get_country_name(row):
    country = row['country_obj']
    if country != None:
        return country.name
    else:
        return None
        
def get_country_code(row):
    country = row['country_obj']
    if country != None:
        return country.alpha_2
    else:
        return None

def get_country_flag(row):
    country = row['country_obj']
    if country != None:
        return country.flag
    else:
        return None
        
kio_df['country_obj'] = kio_df['country'].transform(get_country_object)
kio_df['country'] = kio_df.apply(lambda row: get_country_name(row), axis=1)
kio_df['country_code'] = kio_df.apply(lambda row: get_country_code(row), axis=1)
kio_df['country_flag'] = kio_df.apply(lambda row: get_country_flag(row), axis=1)
kio_df = kio_df.drop('country_obj', axis=1)

ioda_df['country_obj'] = ioda_df['country'].transform(get_country_object)
ioda_df['country'] = ioda_df.apply(lambda row: get_country_name(row), axis=1)
ioda_df['country_code'] = ioda_df.apply(lambda row: get_country_code(row), axis=1)
ioda_df['country_flag'] = ioda_df.apply(lambda row: get_country_flag(row), axis=1)
ioda_df = ioda_df.drop('country_obj', axis=1)

Skipping row for which a country could not be identified: European Union


## Save KIO and IODA data to database. SQL commands will be used for merging the two datasets.

- Save KIO data to a table ("kio")
- Save IODA data to a table ("ioda")

In [ ]:
import sqlite3

conn = sqlite3.connect('../data/ioda_kio.db')
cursor = conn.cursor()

kio_df = kio_df.sort_values("start_date").reset_index(drop=True)
ioda_df = ioda_df.sort_values("start_time").reset_index(drop=True)

cursor.execute("DROP TABLE IF EXISTS kio")

kio_df.to_sql(name='kio', con=conn)

cursor.execute("DROP TABLE IF EXISTS ioda")

ioda_df.to_sql(name='ioda', con=conn);

### Find and create matches table, save to JSON and pkl outputs.

In [ ]:
cursor.execute("DROP TABLE IF EXISTS matched_events;")
query = """
CREATE TABLE matched_events AS 
    SELECT 
        k.[index] as kio_index, ioda.[index] as ioda_index
    FROM 
        kio as k
    INNER JOIN ioda
        ON k.country = ioda.country
            AND k.full_network_shutdown
            AND date(k.start_date, '-1 day') <= ioda.start_time
            AND (ioda.start_time <= date(k.start_date, '+2 day') OR ioda.start_time <= date(k.end_date, '+1 day'))
    
;
"""
# AND (k.geo_scope = 'regional' OR k.geo_scope = 'national')
cursor.execute(query);

import json, pickle

def add_ioda_matches(row):
    return list([x[0] for x in cursor.execute("SELECT ioda_index FROM matched_events WHERE kio_index = "+str(row.name))])
kio_output_df = kio_df.copy()
kio_output_df['ioda_matches'] = kio_output_df.apply(lambda row: add_ioda_matches(row), axis=1)
kio_dict = json.loads(kio_output_df.to_json(orient="records",date_format="iso"))

def add_kio_matches(row):
    return list([x[0] for x in cursor.execute("SELECT kio_index FROM matched_events WHERE ioda_index = "+str(row.name))])
ioda_output_df = ioda_df.copy()
ioda_output_df['kio_matches'] = ioda_output_df.apply(lambda row: add_kio_matches(row), axis=1)
ioda_dict = json.loads(ioda_output_df.to_json(orient="records",date_format="iso"))

with open("../data/ioda_kio_merged.json", "w") as f:
    json.dump({"ioda" : ioda_dict, "kio": kio_dict}, f, indent=4)
    
with open("../data/ioda.pkl", "wb") as f:
    pickle.dump(ioda_output_df, f)
with open("../data/kio.pkl", "wb") as f:
    pickle.dump(kio_output_df, f)

## Print summary of KIO/IODA merged dataset, produce latex table

In [ ]:
 from pprint import pprint

from collections import defaultdict 

shutdown_types = ['full_network_shutdown', 'throttling', 'service_based']
geo_scopes = ["national", "regional", "local"]
network_types = ["broadband_affected", "mobile_affected"]

base_selection = ((kio_output_df['start_date'] >= datetime.datetime(2018,1,1))
                  & (kio_output_df['start_date'] <= datetime.datetime(2021,8,1))
                 )
matched_selection = ((kio_output_df['ioda_matches'].map(len) > 0) 
                     & (kio_output_df['start_date'] >= datetime.datetime(2018,1,1))
                     & (kio_output_df['start_date'] <= datetime.datetime(2021,8,1))
                    )
d = {"total" : len(kio_output_df[base_selection]),
     "count" : len(kio_output_df[matched_selection]),
     "fraction" : len(kio_output_df[matched_selection])/len(kio_output_df[base_selection])}
print("Total: {:.1%} ({} of {})".format(d['fraction'], d['count'], d['total']))

base_selection = ((kio_output_df['start_date'] >= datetime.datetime(2018,1,1))
                  & (kio_output_df['start_date'] <= datetime.datetime(2021,8,1))
                  #& ~kio_output_df['country_code'].str.match("IN")
                 )
matched_selection = ((kio_output_df['ioda_matches'].map(len) > 0) 
                     & (kio_output_df['start_date'] >= datetime.datetime(2018,1,1))
                     & (kio_output_df['start_date'] <= datetime.datetime(2021,8,1))
                     #& ~kio_output_df['country_code'].str.match("IN")
                    )
counts = defaultdict(lambda : defaultdict(dict))

d = {"total" : len(kio_output_df[base_selection]),
     "count" : len(kio_output_df[matched_selection]),
     "fraction" : len(kio_output_df[matched_selection])/len(kio_output_df[base_selection])}
print("Total: {:.1%} ({} of {})".format(d['fraction'], d['count'], d['total']))

counts["All"]["All"]["BB+M"] = d

for network_type in network_types:
    total = len(kio_output_df[base_selection
                              & kio_output_df[network_type]
                             ])
    match_count = len(kio_output_df[matched_selection
                                    & kio_output_df[network_type] 
                                   ])
    d = {"total" : total,
         "count" : match_count,
         "fraction" : match_count/total}
    counts["All"]["All"][network_type] = d
    print("\t\t{} {:.1%} ({} of {})".format(network_type, d['fraction'], d['count'], d['total']))

for geo_scope in geo_scopes:
    total = len(kio_output_df[base_selection
                              & kio_output_df['geo_scope'].str.match(geo_scope)
                             ])
    match_count = len(kio_output_df[matched_selection
                                    & kio_output_df['geo_scope'].str.match(geo_scope)
                                   ])
    d = {"total" : total,
         "count" : match_count,
         "fraction" : match_count/total}
    counts[geo_scope]["All"]["BB+M"] = d
    print("\t{} {:.1%} ({} of {})".format(geo_scope, d['fraction'], d['count'], d['total']))

    for network_type in network_types:
        total = len(kio_output_df[base_selection
                                  & kio_output_df['geo_scope'].str.match(geo_scope)
                                  & kio_output_df[network_type]
                                 ])
        match_count = len(kio_output_df[matched_selection
                                        & kio_output_df['geo_scope'].str.match(geo_scope)
                                        & kio_output_df[network_type] 
                                       ])
        d = {"total" : total,
             "count" : match_count,
             "fraction" : match_count/total}
        counts[geo_scope]["All"][network_type] = d
        print("\t\t{} {:.1%} (total: {} count: {})".format(network_type, d['fraction'], d['count'], d['total']))

for shutdown_type in shutdown_types:
    total = len(kio_output_df[base_selection & kio_output_df[shutdown_type]])
    match_count = len(kio_output_df[matched_selection 
                                    & kio_output_df[shutdown_type]])
    d = {"total" : total,
         "count" : match_count,
         "fraction" : match_count/total}
    counts["All"][shutdown_type]["BB+M"] = d
    print("{}\n\tAll: {:.1%} ({} of {})".format(shutdown_type, d['fraction'], d['count'], d['total']))
    for network_type in network_types:
        total = len(kio_output_df[base_selection
                                  & kio_output_df[shutdown_type] 
                                  & kio_output_df[network_type]
                                 ])
        match_count = len(kio_output_df[matched_selection
                                        & kio_output_df[shutdown_type] 
                                        & kio_output_df[network_type] 
                                       ])
        d = {"total" : total,
             "count" : match_count,
             "fraction" : match_count/total}
        counts["All"][shutdown_type][network_type] = d
        print("\t\t{}: {:.1%} ({} of {})".format(network_type, d['fraction'], d['count'], d['total']))
    for geo_scope in geo_scopes:
        total = len(kio_output_df[base_selection
                                  & kio_output_df[shutdown_type] 
                                  & kio_output_df['geo_scope'].str.match(geo_scope)
                                 ])
        match_count = len(kio_output_df[matched_selection
                                        & kio_output_df[shutdown_type] 
                                        & kio_output_df['geo_scope'].str.match(geo_scope)
                                       ])
        d = {"total" : total,
             "count" : match_count,
             "fraction" : match_count/total}
        counts[geo_scope][shutdown_type]["BB+M"] = d
        print("\t{} {:.1%} ({} of {})".format(geo_scope, d['fraction'], d['count'], d['total']))
        
        for network_type in network_types:
            total = len(kio_output_df[base_selection
                                      & kio_output_df[shutdown_type] 
                                      & kio_output_df['geo_scope'].str.match(geo_scope)
                                      & kio_output_df[network_type]
                                     ])
            match_count = len(kio_output_df[matched_selection
                                            & kio_output_df[shutdown_type] 
                                            & kio_output_df['geo_scope'].str.match(geo_scope)
                                            & kio_output_df[network_type] 
                                           ])
            d = {"total" : total,
                 "count" : match_count,
                 "fraction" : match_count/total}
            counts[geo_scope][shutdown_type][network_type] = d
            print("\t\t{} {:.1%} (total: {} count: {})".format(network_type, d['fraction'], d['count'], d['total']))
#kio_df[kio_df['full_network_shutdown']]



substitutions = {"national" : "National",
                 "regional" : "Regional",
                 "local" : "Local",
                 "broadband_affected" : "Broadband affected",
                 "mobile_affected" : "Mobile affected",
                 "full_network_shutdown" : "Full network shutdown",
                 "service_based" : "App-specific/Service-based",
                 "throttling" : "Throttling"}
scopes = ["All", "national", "regional", "local"]
networks = ['BB+M', 'broadband_affected', 'mobile_affected']
shutdown_types = ["All", 'full_network_shutdown', 'service_based', 'throttling']

#pprint(counts)
table_str = (
    """\\begin{table*}[t]
    \\caption{Multi-row table}
    \\begin{center}
    \\begin{tabular}{c|ccc|ccc|ccc|ccc}
        \\hline
    """)

table_str += ("\t & " + " & ".join(["\\multicolumn{3}{c|}{All}", 
                                  "\\multicolumn{3}{c|}{National}",
                                  "\\multicolumn{3}{c|}{Regional}",
                                  "\\multicolumn{3}{c}{Local}"])+ " \\\\ \n")
table_str += " & ".join(["\t"]+['BB+M', "BB", "M"]*4)+ " \\\\ \\hline \n"

for shutdown_type in shutdown_types:
    #print(scope)
    if shutdown_type in substitutions:
        label = substitutions[shutdown_type]
    else:
        label = shutdown_type
    row_list = [label]
    for scope in scopes:
        #print("\t",network)
        for network in networks:
            #print("\t\t",shutdown_type)
            row_list.append("{:.1f}\\%".format(counts[scope][shutdown_type][network]['fraction']*100))
                
    table_str += "\t"+" & ".join(row_list)+" \\\\\n"

table_str += """        \\hline
    \\end{tabular}
    \\end{center}
    \\label{tab:multicol}
\end{table*}
    """
print(table_str)
from IPython.display import Latex
#Latex(table_str)

Total: 8.4% (55 of 657)
Total: 8.4% (55 of 657)
		broadband_affected 13.3% (43 of 324)
		mobile_affected 8.6% (54 of 630)
	national 27.2% (46 of 169)
		broadband_affected 28.0% (total: 40 count: 143)
		mobile_affected 28.7% (total: 45 count: 157)
	regional 4.7% (7 of 149)
		broadband_affected 5.4% (total: 3 count: 56)
		mobile_affected 4.9% (total: 7 count: 144)
	local 0.6% (2 of 331)
		broadband_affected 0.0% (total: 0 count: 123)
		mobile_affected 0.6% (total: 2 count: 322)
full_network_shutdown
	All: 10.1% (55 of 544)
		broadband_affected: 17.4% (43 of 247)
		mobile_affected: 10.1% (54 of 533)
	national 51.7% (46 of 89)
		broadband_affected 51.9% (total: 40 count: 77)
		mobile_affected 51.1% (total: 45 count: 88)
	regional 5.3% (7 of 132)
		broadband_affected 5.9% (total: 3 count: 51)
		mobile_affected 5.4% (total: 7 count: 130)
	local 0.6% (2 of 315)
		broadband_affected 0.0% (total: 0 count: 117)
		mobile_affected 0.6% (total: 2 count: 308)
throttling
	All: 10.8% (7 of 65)
		broad

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=2b9a2535-ed0e-48ec-8277-3dc0993a1e10' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>